# Imports, Initialization, and Hyperparameters

In [1]:
import pandas as pd
import torch
import numpy as np
import math
import torch_geometric

C:\Users\colef\AppData\Local\Temp\ipykernel_29028\3082682486.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#Import Rao and Lieberman data
rao_data = pd.read_csv(r"C:\Users\colef\OneDrive - University of Miami\Documents\GitHub\Graph-Compression\Data\hg38_GM12878_Rao_2014-raw_TADs_with_gene_ids.tsv", sep="\t")
rao_data.drop(['chr','start','end'],axis=1,inplace=True)
lieberman_data = pd.read_csv(r"C:\Users\colef\OneDrive - University of Miami\Documents\GitHub\Graph-Compression\Data\hg38_GM12878_Lieberman_2009-raw_TADs_with_gene_ids.tsv", sep="\t")
lieberman_data.drop(['chr','start','end'],axis=1,inplace=True)

#Process data
def get_genes(data):
    data = data.to_numpy()
    data = data.flatten()
    data = data[~pd.isnull(data)]
    data = list(set(data))
    for i,gene in enumerate(data):
        data[i] = gene.split('.', 1)[0]
    return data

rao_genes = get_genes(rao_data)
lieberman_genes = get_genes(lieberman_data)
len(lieberman_genes), len(rao_genes)

(50295, 56750)

In [3]:
#Import PPI Data and construct protein list
data = pd.read_csv(r"C:\Users\colef\OneDrive - University of Miami\Documents\GitHub\Graph-Compression\Data\HI-union.tsv",sep='\t')
def get_proteins(data,genes):
    output_data = data.drop(data[(data['Protein1'] == data['Protein2']) | (~np.isin(data['Protein1'],genes)) | (~np.isin(data['Protein2'],genes))].index)
    output_data.reset_index(drop=True,inplace=True)
    proteins = set([*output_data.Protein1,*output_data.Protein2])
    proteins = list(proteins)
    return output_data, proteins

Ldata, lieberman_proteins = get_proteins(data,lieberman_genes)
Rdata, rao_proteins = get_proteins(data,rao_genes)

print('Lieberman','Rao')
print(len(lieberman_proteins), len(rao_proteins))
print(Ldata.shape, Rdata.shape)

Lieberman Rao
6944 8312
(40612, 2) (54892, 2)


In [4]:
#Check for overlap between genes and proteins
temp = []
for gene in lieberman_genes:
    if gene in lieberman_proteins:
        temp.append(gene)
lieberman_genes = temp

temp = []
for gene in rao_genes:
    if gene in rao_proteins:
        temp.append(gene)
rao_genes = temp

print(len(lieberman_genes), len(rao_genes))
print(len(lieberman_proteins), len(rao_proteins))

6944 8312
6944 8312


In [5]:
#Calculate compression ratio
def process(gene,protein_list):
    gene = str(gene)
    gene = gene.split('.', 1)[0]
    if gene in protein_list:
        return gene
    else:
        return np.nan

rao_data = rao_data.map(process, protein_list=rao_proteins)
lieberman_data = lieberman_data.map(process, protein_list=lieberman_proteins)

rao_data = rao_data.dropna(how='all')
lieberman_data = lieberman_data.dropna(how='all')

In [6]:
#Construct Edge List
def construct_edge_list(data,protein_list):
    edge_list = torch.tensor([data['Protein1'].apply(lambda x: protein_list.index(x)).values,data['Protein2'].apply(lambda x: protein_list.index(x)).values],dtype=torch.long)
    edge_list = torch_geometric.utils.to_undirected(edge_list)
    return edge_list

lieberman_edge_list = construct_edge_list(Ldata,lieberman_proteins)
rao_edge_list = construct_edge_list(Rdata,rao_proteins)

#Contruct Adjacency Matrix from Edge List
def construct_adjacency_matrix(edge_list,protein_list):
    A = torch.zeros(len(protein_list),len(protein_list))
    for i in range(edge_list.shape[1]):
        A[edge_list[0,i],edge_list[1,i]] = 1
        A[edge_list[1,i],edge_list[0,i]] = 1
    return A
lieberman_A = construct_adjacency_matrix(lieberman_edge_list,lieberman_proteins)
rao_A = construct_adjacency_matrix(rao_edge_list,rao_proteins)

#Construct Degree Matrix from Adjacency Matrix
def construct_degree_matrix(A):
    D = torch.zeros(len(A),len(A))
    for i in range(len(A)):
        D[i,i] = A[i].sum()
    return D
lieberman_D = construct_degree_matrix(lieberman_A)
rao_D = construct_degree_matrix(rao_A)

#Define Max Degree
lieberman_max = lieberman_D.max().item()
rao_max = rao_D.max().item()
print(lieberman_max,rao_max)

C:\Users\colef\AppData\Local\Temp\ipykernel_29028\3301662629.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  edge_list = torch.tensor([data['Protein1'].apply(lambda x: protein_list.index(x)).values,data['Protein2'].apply(lambda x: protein_list.index(x)).values],dtype=torch.long)


517.0 604.0


In [7]:
##Hyperparameters

#K: Maximum hop distance (K = 2)
K = 2

#Gamma: Hop discount factor (γ = 0.01)
gamma = 0.01

#Eta: Node degree threshold (η = 15)
eta = 15

#Phi: Compression ratio (φ = 0.2)
rao_phi = 1 - (rao_data.shape[0] / len(rao_genes)) #Rao
lieberman_phi = 1 - (lieberman_data.shape[0] / len(lieberman_genes)) #Lieberman
print('Rao',rao_phi)
print('Lieberman',lieberman_phi)

#m: Hidden layer size (m = log2(max_degree))
m = int(math.log2(lieberman_max)) + 1 #It is the same max for both datasets

#p: Embedding layer size (p = 2m)
p = 2*m

Rao 0.6856352261790183
Lieberman 0.838565668202765


# Feature Extraction

In [8]:
def extract_features(protein_list,edge_list,D):
    features = torch.zeros(len(protein_list),m,dtype=torch.float32)
    k = 1

    while k <= K:
        for protein in range(len(protein_list)):
            #Get k-hop neighbors of protein
            neighbors = torch_geometric.utils.k_hop_subgraph(protein,k,edge_list)[0]

            #Get temporary features using neighbor degrees
            temporary_features = torch.zeros([m])
            for neighbor in neighbors:
                idx = int(math.log2(D[neighbor,neighbor].item()))
                temporary_features[idx] += 1

            #Apply hop discount factor and add to features
            temporary_features = pow(gamma,k-1)*temporary_features
            features[protein] += temporary_features
        k += 1
    return features

lieberman_features = extract_features(lieberman_proteins,lieberman_edge_list,lieberman_D)
rao_features = extract_features(rao_proteins,rao_edge_list,rao_D)
print(lieberman_features.shape,rao_features.shape)

torch.Size([6944, 10]) torch.Size([8312, 10])


# Embedding Learning

In [9]:
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        
        #GCN Layers
        self.conv1 = GCNConv(m, m, cached=True)
        nn.init.uniform_(self.conv1.lin.weight,-math.sqrt(6)/math.sqrt(m+m),math.sqrt(6)/math.sqrt(m+m)) #Initialize weights
        self.conv2 = GCNConv(m, p, cached=True)
        nn.init.uniform_(self.conv2.lin.weight,-math.sqrt(6)/math.sqrt(p+m),math.sqrt(6)/math.sqrt(p+m)) #Initialize weights
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.tanh(x)
        x = self.conv2(x, edge_index)
        x = F.tanh(x)
        return x

In [10]:
#Put model and data on the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
lieberman_features.to(device)
rao_features.to(device)

#Use model to get embeddings
lieberman_embeddings = model(lieberman_features,lieberman_edge_list)
rao_embeddings = model(rao_features,rao_edge_list)
print(lieberman_embeddings.shape,rao_embeddings.shape)

torch.Size([6944, 20]) torch.Size([8312, 20])


# Guiding List

In [11]:
#Reduce size of protein list based on degree threshold η and store features of reduced proteins
def reduce_embeddings(embeddings,D,protein_list,eta):
    reduced_protein_list = []
    reduced_features = []
    for i in range(len(protein_list)):
        if D[i,i] >= eta:
            reduced_protein_list.append(protein_list[i])
            reduced_features.append(embeddings[i].cpu().detach().numpy())
    return reduced_protein_list, reduced_features

reduced_lieberman_proteins, reduced_lieberman_embeddings = reduce_embeddings(lieberman_embeddings,lieberman_D,lieberman_proteins,eta)
reduced_rao_proteins, reduced_rao_embeddings = reduce_embeddings(rao_embeddings,rao_D,rao_proteins,eta)
print(len(reduced_lieberman_proteins),len(reduced_rao_proteins))

1401 1826


In [12]:
#Initialize list of norms and sort protein list based on norms
def sort_proteins(reduced_features,reduced_protein_list):
    norms = []
    for i in range(len(reduced_features)):
        norms.append(np.linalg.norm(reduced_features[i]))

    sorted_protein_list = [x for _,x in sorted(zip(norms,reduced_protein_list),reverse=True)]
    return sorted_protein_list

sorted_lieberman_proteins = sort_proteins(reduced_lieberman_embeddings,reduced_lieberman_proteins)
sorted_rao_proteins = sort_proteins(reduced_rao_embeddings,reduced_rao_proteins)
print(len(sorted_lieberman_proteins),len(sorted_rao_proteins))

1401 1826


# Graph Compression

In [13]:
#Get neighbors using adjacency matrix
def get_Neighbors(protein_idx,A):
    neighbors = []
    for i in range(len(A)):
        if A[protein_idx,i] == 1:
            neighbors.append(i)
    return neighbors

In [14]:
def compress(protein_list,sorted_protein_list,edge_list,A,D,phi,max_degree):
    #Initialize compress_rate and make deep copies of protein list
    compress_rate = 0
    compressed_A = A.clone()
    compressed_D = D.clone()
    compressed_edge_list = edge_list.clone()
    compressed_protein_list = protein_list.copy()
    guided_list = sorted_protein_list.copy()

    #Compress loop
    while compress_rate < phi:
        
        #Iterate through guided list:
        for protein in guided_list:
            
            #Get minimum degree of neighbors
            protein_idx = compressed_protein_list.index(protein)
            neighbors = get_Neighbors(protein_idx,compressed_A)
            min_degree = max_degree
            for neighbor_idx in neighbors:
                neighbor_degree = compressed_D[neighbor_idx,neighbor_idx].item()
                min_degree = min(min_degree,neighbor_degree)
            
            #Make list of nodes for compression based on minimum degree
            compress_list = []
            for neighbor_idx in neighbors:
                neighbor_degree = compressed_D[neighbor_idx,neighbor_idx].item()
                if neighbor_degree == min_degree:
                    compress_list.append(neighbor_idx)
            compress_list.append(protein_idx)
            
            #Compress nodes in compress list
            if len(compress_list) > 1:
                supernode = '+'.join([compressed_protein_list[i] for i in compress_list])
                
                supernode_neighbors = set()
                
                #Store supernode neighbors
                for subnode_idx in compress_list:
                    supernode_neighbors = supernode_neighbors | set(get_Neighbors(subnode_idx,compressed_A))
                
                ##Update Adjacency Matrix
                
                #Add Supernode and connections
                compressed_A = torch.cat([compressed_A,torch.zeros(1,len(compressed_A))])
                compressed_A = torch.cat([compressed_A,torch.zeros(len(compressed_A),1)],dim=1)
                
                for neighbor in supernode_neighbors:
                    compressed_A[-1,neighbor] = 1
                    compressed_A[neighbor,-1] = 1
                    
                #Remove subnodes
                for subnode_idx in sorted(compress_list,reverse=True):
                    compressed_A = torch.cat([compressed_A[:subnode_idx],compressed_A[subnode_idx+1:]])
                    compressed_A = torch.cat([compressed_A[:,:subnode_idx],compressed_A[:,subnode_idx+1:]],dim=1)
                
                ##Update Degree Matrix
                compressed_D = construct_degree_matrix(compressed_A)
                
                ##Update Guided List
                for subnode_idx in compress_list:
                    if compressed_protein_list[subnode_idx] in guided_list:
                        guided_list.remove(compressed_protein_list[subnode_idx])
                guided_list.append(supernode)
                
                ##Update Protein List
                compressed_protein_list.append(supernode)
                for subnode_idx in sorted(compress_list,reverse=True):
                    compressed_protein_list.remove(compressed_protein_list[subnode_idx])
                

            compress_rate = 1 - len(compressed_protein_list)/len(protein_list)
            print(compress_rate)
            if compress_rate >= phi:
                return compressed_protein_list,compressed_A,compressed_D

compressed_lieberman_proteins,compressed_lieberman_A,compressed_lieberman_D, = compress(lieberman_proteins,sorted_lieberman_proteins,lieberman_edge_list,lieberman_A,lieberman_D,lieberman_phi,lieberman_max)
compressed_rao_proteins,compressed_rao_A,compressed_rao_D, = compress(rao_proteins,sorted_rao_proteins,rao_edge_list,rao_A,rao_D,rao_phi,rao_max)

0.0018721198156681496
0.0036002304147465525
0.003888248847926268
0.005328341013824844
0.007056451612903247
0.007344470046082963
0.008496543778801824
0.00936059907834097
0.010224654377880227
0.010800691244239657
0.011520737327188946
0.013248847926267238
0.01382488479262678
0.01497695852534564
0.017281105990783363
0.01742511520737322
0.023329493087557607
0.02376152073732718
0.025201612903225756
0.025345622119815614
0.0267857142857143
0.027217741935483875
0.027793778801843305
0.02808179723502302
0.028369815668202736
0.031538018433179715
0.03254608294930872
0.03312211981566815
0.03326612903225812
0.03427419354838712
0.03456221198156684
0.034706221198156695
0.035282258064516125
0.03542626728110598
0.0367223502304147
0.037154377880184386
0.037298387096774244
0.038450460829493105
0.03859447004608296
0.03931451612903225
0.039746543778801824
0.04003456221198154
0.04046658986175111
0.040898617511520685
0.04118663594470051
0.04133064516129037
0.041618663594470084
0.04176267281105994
0.04190668202

In [18]:
#Save compressed protein list
np.savetxt(r"C:\Users\colef\OneDrive - University of Miami\Documents\GitHub\Graph-Compression\Data\HI-union_compressedLieberman.csv",compressed_lieberman_proteins,fmt='%s')
np.savetxt(r"C:\Users\colef\OneDrive - University of Miami\Documents\GitHub\Graph-Compression\Data\HI-union_compressedRao.csv",compressed_rao_proteins,fmt='%s')